# 데이터 가공

## 웹 크롤링해온 이미지 확인 및 경로 설정

In [18]:
import cv2
import os
import shutil
path_dir = './dataset/'

folder_list = os.listdir(path_dir)
cur_path = None
print(len(folder_list))
print(folder_list)

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

232
['강동원', '강민경', '강세련', '강형욱', '강호동', '고소영', '고준희', '공유', '곽도원', '구혜선', '권나라', '권상우', '권정열', '기리보이', '김가영', '김경호', '김고은', '김남길', '김다미', '김민주', '김민희', '김병만', '김새론', '김성균', '김세연', '김소현', '김수현', '김용만', '김우빈', '김유정', '김윤석', '김정현', '김준호', '김지민', '김지원', '김태리', '김태원', '김태희', '김향기', '김현수', '김혜수', '김희선', '김희철', '남규리', '남주혁', '노윤서', '노지선', '더콰이엇', '데프콘', '디노', '로이킴', '류승범', '류준열', '문근영', '문세윤', '문채원', '미연', '민경훈', '민효린', '박규영', '박명수', '박보검', '박보영', '박서준', '박소담', '박수진', '박신혜', '박연경', '박완규', '박중훈', '박지윤', '박해수', '박형식', '배정남', '백예린', '백종원', '버논', '범규', '뷔', '빈지노', '서강준', '서지혜', '설인아', '성시경', '소지섭', '손석구', '손예진', '손태영', '손호준', '송강', '송강호', '송승헌', '송중기', '송하영', '송혜교', '수지', '슈가', '스윙스', '신동엽', '신민아', '신세경', '신아영', '심은경', '심혜진', '아린', '아이유', '안보현', '안성기', '안유진', '안효섭', '양다일', '양세찬', '양세형', '에일리', '연정훈', '연준', '영훈', '예지', '오나라', '오나미', '오달수', '오연서', '원빈', '원우', '원진아', '윈터', '유나', '유상무', '유세윤', '유아인', '유연석', '유재석', '유해진', '윤계상', '윤아', '윤여정', '윤종신', '윤태진', '은지원', '이나영', '이나은', '이도현', '이동욱', '이민기', '이민정

## 이미지를 얼굴만 보이도록 자르기

In [2]:
import numpy as np
import glob

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [3]:
folderNum = 0
for folder in folder_list:
    picture_list = path_dir + folder + '/'
    imgNum  = 0
    if not os.path.isdir('./custom_dataset/'+ str(folderNum)): # 경로에 한글이 들어가면 imwrite에서 오류가 남
        os.makedirs('./custom_dataset/'+ str(folderNum)) # 숫자폴더 생성
    for picture in os.listdir(picture_list):
        ff = np.fromfile(picture_list + picture, np.uint8)
        img = cv2.imdecode(ff,cv2.IMREAD_UNCHANGED) # 한글경로 실행법
        if img is not None:
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 흑백사진을 확인할 수 있게
            gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
            try :
                faces = face_cascade.detectMultiScale(gray, 1.3,5) # 얼굴인식 불가능시 건너뛰기
            except :
                pass
            for (x,y,w,h) in faces:
                cropped = img[y - int(h / 4):y + h + int(h / 4), x - int(w / 4):x + w + int(w / 4)]
                if cropped.size != 0:
                    cv2.imwrite('./custom_dataset/'+ str(folderNum) + '/image_' + str(imgNum) + '.jpg', cropped) # 숫자폴더에 얼굴만 자른 사진 생성
                else :
                    pass
                imgNum += 1
        else:
            print('imread failed',picture_list + picture)
    os.rename('./custom_dataset/'+ str(folderNum) , './custom_dataset/'+folder) # 숫자폴더를 다시 연예인 이름으로 변경
    folderNum += 1

None ./dataset/강호동/image_3.jpg
None ./dataset/한효주/image_5.jpg


#### 데이터를 얼굴만 분리
#### opencv의 한계로 이상한 사진이 섞여들어가거나 분리가 잘못된 파일이 생성됨 
#### 손으로 직접 분리, 이상한 데이터는 삭제, 부족한 데이터는 따로 구해서 집어넣었음 (최소50장)

## train, test 폴더 생성 및 분류

In [15]:
train_directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

# train,test 디렉토리 만들기
for directory in train_directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# train, test 데이터 분리 함수
def dataset_split(query, test_cnt, folder_name):
    for directory in train_directory_list:
        if not os.path.isdir(directory + '/' + folder_name):
            os.makedirs(directory + '/' + folder_name)
    cnt = 0
    print(query)
    for file_name in os.listdir(query):
        if cnt < test_cnt:
            print(f'[Test Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/test/' +folder_name+ '/' + file_name)
        else:
            print(f'[Train Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/train/' +folder_name+ '/' + file_name)
        cnt += 1
    shutil.rmtree(query)

In [16]:
custom_img_path = './custom_dataset/' # custom image path 설정

In [25]:
for i in folder_list:
    picture_list = custom_img_path+i # 분류한 데이터가 최소 50개이므로 test 를 10개로)
    dataset_split(picture_list, 10, i) # (datasetPATH, test갯수, 연예인이름)

./custom_dataset/김태리
[Test Dataset] image_0.jpg
[Test Dataset] image_1.jpg
[Test Dataset] image_10.jpg
[Test Dataset] image_11.jpg
[Test Dataset] image_12.jpg
[Test Dataset] image_13.jpg
[Test Dataset] image_14.jpg
[Test Dataset] image_15.jpg
[Test Dataset] image_16.jpg
[Test Dataset] image_17.jpg
[Train Dataset] image_18.jpg
[Train Dataset] image_19.jpg
[Train Dataset] image_2.jpg
[Train Dataset] image_20.jpg
[Train Dataset] image_21.jpg
[Train Dataset] image_22.jpg
[Train Dataset] image_23.jpg
[Train Dataset] image_24.jpg
[Train Dataset] image_25.jpg
[Train Dataset] image_26.jpg
[Train Dataset] image_27.jpg
[Train Dataset] image_28.jpg
[Train Dataset] image_29.jpg
[Train Dataset] image_3.jpg
[Train Dataset] image_30.jpg
[Train Dataset] image_31.jpg
[Train Dataset] image_32.jpg
[Train Dataset] image_33.jpg
[Train Dataset] image_34.jpg
[Train Dataset] image_35.jpg
[Train Dataset] image_36.jpg
[Train Dataset] image_37.jpg
[Train Dataset] image_38.jpg
[Train Dataset] image_39.jpg
[Train 